In [2]:
import pandas as pd
import numpy as np

In [9]:
data = np.array([
        ['','P0','P1','P2','P3','P4','P5','P6','P7','P8','P9','P10','P11','P12'], 
        ['Pink Floyd',1,1,1,1,1,1,1,1,1,1,1,1,1],
        ['The Beatles',0,1,1,0,0,1,1,1,1,1,1,1,1],
        ['R.E.M',0,0,1,1,0,1,0,1,1,1,1,0,0],
        ['Nirvana',1,1,0,0,0,0,0,0,1,0,0,1,1],
        ['Queen',1,1,0,0,1,0,1,0,1,1,1,1,0],
        ['Oasis',1,0,1,1,0,0,1,1,0,0,1,0,0],
        ['Class','J','J','J','J','J','J','A','A','A','A','A','A','A']])
dF=pd.DataFrame(data=data[1:,1:],
                  index=data[1:,0],
                  columns=data[0,1:])
dF.loc['Class']['P1']
for k in dF.index[:-1]:
    print(k)
dF.loc['Queen']['P0']
dF

Pink Floyd
The Beatles
R.E.M
Nirvana
Queen
Oasis


,P0,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12
Pink Floyd,1,1,1,1,1,1,1,1,1,1,1,1,1
The Beatles,0,1,1,0,0,1,1,1,1,1,1,1,1
R.E.M,0,0,1,1,0,1,0,1,1,1,1,0,0
Nirvana,1,1,0,0,0,0,0,0,1,0,0,1,1
Queen,1,1,0,0,1,0,1,0,1,1,1,1,0
Oasis,1,0,1,1,0,0,1,1,0,0,1,0,0
Class,J,J,J,J,J,J,A,A,A,A,A,A,A


In [26]:
# Cálculo de P(band|clase) 
def prob_of_band_given_class(band,cl):
    Nc=0
    nc_band=0
    for k in dF.columns:
        if dF.loc['Class'][k] == cl:
            Nc += 1
            if dF.loc[band][k] == '1':
                nc_band += 1
    return nc_band/Nc

# Cálculo de P(clase)
def prob_of_class(cl):
    counter=0
    for k in dF.columns:
        if dF.loc['Class'][k] == cl:
            counter +=1
    return counter/len(dF.columns)

print(len(dF.columns))
prob_of_class('A')

# Cálculo de P(clase|persona), la probabilidad de que una persona dada pertenezca a la clase
def prob_of_being_of_class(cl,person):
    p=0
    p= prob_of_class(cl)
    for k in dF.index[:-1]:
        if dF.loc[k][person] == '1':
            p *= prob_of_band_given_class(k,cl)
        else:
            p *= (1-prob_of_band_given_class(k,cl))
    return p

prob_of_being_of_class('J','P12')

13


0.009615384615384616

In [46]:
# Clasificador bayesiano ingenuo MEV. Clasifica un individuo como 'A' (adulto) o 'J' (joven)
def mve_nb_classifier1(person):
    if prob_of_being_of_class('A',person) < prob_of_being_of_class('J',person):
        return 'J'
    if prob_of_being_of_class('J',person) < prob_of_being_of_class('A',person):
        return 'A'
    else:
        if np.random(0,1) < 0.5:
            return 'A'
        else:
            return 'J'
        
i = 0
for k in dF.columns:
    print('P',i,' ',mve_nb_classifier1(k),dF.loc['Class'][k])
    i +=1

P 0   J J
P 1   A J
P 2   A J
P 3   J J
P 4   J J
P 5   A J
P 6   A A
P 7   A A
P 8   A A
P 9   A A
P 10   A A
P 11   A A
P 12   A A


In [65]:
# Datos para predicciones
tdata = np.array([
        ['','Pink Floyd','The Beatles', 'R.E.M.','Nirvana','Queen','Oasis'],
        ['x1',1, 1, 0, 1, 1, 0],
        ['x2',1, 0, 1, 1, 1, 1],
        ['x3',1, 1, 0, 0, 0, 0],
        ['x4',1, 1, 1, 1, 1, 1],
        ['x5',0, 1, 1, 1, 1, 1]])

testdF=pd.DataFrame(data=tdata[1:,1:],
                  index=tdata[1:,0],
                  columns=tdata[0,1:])
testdF.loc['x1']

Pink Floyd     1
The Beatles    1
R.E.M.         0
Nirvana        1
Queen          1
Oasis          0
Name: x1, dtype: object

In [68]:
# Predicciones
for k in testdF.index:
    print(mve_nb_classifier1(testdF.loc[k]))

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().